# UW-Madison GI Tract Image Segmentation

## Check model performance

In [2]:
#!/usr/bin/env python

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.options.plotting.backend = "plotly"
from tqdm.notebook import tqdm
tqdm.pandas()
import numpy as np
import os
import re
import cv2
from plotly.offline import init_notebook_mode
from glob import glob

import tensorflow as tf

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
from matplotlib.colors import LinearSegmentedColormap
import plotly
import PIL
import cv2

# Import Tensorflow and Keras
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

from keras.models import load_model


		– MATPLOTLIB VERSION: 3.5.2


In [3]:
%cd 'hpc_train_files'

/Users/viktor/Documents/BA/GIT/HealthyOrganTracker/hpc_train_files


In [4]:
from utility import rle_encode, rle_decode, open_gray16
from config import CFG
from dataloader import DataGenerator
from loss import dice_coef, iou_coef, dice_loss, bce_dice_loss, dice_coef2
from datapreparation import extract_metadata, remove_faulties

## Config ⚙️


In [5]:
BASE_PATH = 'input/uw-madison-gi-tract-image-segmentation/'
TRAIN_DIR =  BASE_PATH +'train'
TRAIN_CSV =  BASE_PATH +'train.csv'
STYE = "multilabel"
DEBUG = True

"""
Backbones available: 
        'efficientnetb0'
        'efficientnetb1'
        'efficientnetb2'
        'efficientnetb3'
        'efficientnetb4'
        'efficientnetb5'
        'efficientnetb6'
        'efficientnetb7'
        'inceptionresnetv2'
        'inceptionv3'
        'resnet50'
        'resnext50'
        'resnext101'
        'seresnext50'
        'seresnext101'
        'densenet121'
        'densenet201'
        """

"\nBackbones available: \n        'efficientnetb0'\n        'efficientnetb1'\n        'efficientnetb2'\n        'efficientnetb3'\n        'efficientnetb4'\n        'efficientnetb5'\n        'efficientnetb6'\n        'efficientnetb7'\n        'inceptionresnetv2'\n        'inceptionv3'\n        'resnet50'\n        'resnext50'\n        'resnext101'\n        'seresnext50'\n        'seresnext101'\n        'densenet121'\n        'densenet201'\n        "

In [6]:
cfg = CFG(
    backbone            = 'efficientnetb0',
    img_dims            = (256,256,3),
    model               = 'U-Net',
    batch_size          = 16, 
    epochs              = 1, 
    kaggle              = False, 
    use_fold_csv        = True,
    semi3d_data         = False,
    remove_faulty_cases = True,
    use_crop_data       = False)

## Load prepared Dataframe 

In [7]:
DEBUG = False
if DEBUG:
    df_train["multilabel_mask_path"] = df_train.progress_apply(lambda _row: make_seg_mask(_row, outputdir, resize_to=SEG_SHAPE, resize=False), axis=1)
    df_train.to_csv("df_train_with_mask_paths.csv", index=False)
else:
    df_train = pd.read_csv("df_train_with_mask_paths.csv")
    df_train.fillna('', inplace=True)



In [8]:
df_train

,Unnamed: 0,id,large_bowel,small_bowel,stomach,path,case,day,slice,width,...,count,path00,path01,path02,image_paths,rs,re,cs,ce,multilabel_mask_path
0,0,case123_day20_slice_0001,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,1,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
1,1,case123_day20_slice_0002,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,2,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
2,2,case123_day20_slice_0003,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,3,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
3,3,case123_day20_slice_0004,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,4,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
4,4,case123_day20_slice_0005,,,,input/uw-madison-gi-tract-image-segmentation/t...,123,20,5,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,20,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38491,38491,case30_day0_slice_0140,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,140,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
38492,38492,case30_day0_slice_0141,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,141,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
38493,38493,case30_day0_slice_0142,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,142,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...
38494,38494,case30_day0_slice_0143,,,,input/uw-madison-gi-tract-image-segmentation/t...,30,0,143,266,...,0,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,input/uw-madison-gi-tract-image-segmentation/t...,['input/uw-madison-gi-tract-image-segmentation...,1,10000,0,266,input/uw-madison-gi-tract-image-segmentation/m...


In [9]:
for file in glob(os.path.join("../evaluation_results/efb2", "**"), recursive=True):
    if file.endswith(".csv"):
        print(file)

../evaluation_results/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_False_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2034_FOLD_3history.csv
../evaluation_results/efb2/UNet_BB_efficientnetb2_DIM_(288, 288, 3)_SEMI3D_True_CROPDATA_False_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2024_FOLD_3history.csv
../evaluation_results/efb2/UNet_BB_efficientnetb2_DIM_(288, 288, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2024_FOLD_3history.csv
../evaluation_results/efb2/UNet_BB_efficientnetb2_DIM_(288, 288, 3)_SEMI3D_True_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2024_FOLD_3history.csv
../evaluation_results/efb2/UNet_BB_efficientnetb2_DIM_(288, 288, 3)_SEMI3D_False_CROPDATA_True_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2021_FOLD_3history.csv
../evaluation_results/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-

In [10]:
def get_performance_from_history_df(path, output="all_histories.csv", extra_regex=False):
    if not extra_regex:
        all_histories = pd.DataFrame(columns=['Encoder','Größe' ,'Beste Epoche', 'Loss', 'Val. loss', 'Dice', 'Val. Dice', 'IoU', 'Val. IoU'])
    else:
        all_histories = pd.DataFrame(columns=['Encoder','Größe' ,'Beste Epoche', 'Loss', 'Val. loss', 'Dice', 'Val. Dice', 'IoU', 'Val. IoU', '2.5D', 'Crop' , 'Bereinigt', 'Batch Size', 'Epochen'])

    # Iterate over every historyFold3.csv in tensorboard_logs/base_BS_16_no_add_ons/subfolder and plot each column values with plotly and mark the maximum value and drop the first column
    for file in glob(os.path.join(path, "**"), recursive=True):
        if file.endswith(".csv"):
            print(file)
            history_df = pd.read_csv(file, index_col=0)
            
            regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties = get_meta_info_from_model_path(file)
            
            # print all regex parameters 
            print(regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties)

            best_epoch = get_best_epoch(history_df)

            loss = history_df.loc[best_epoch, 'loss']
            val_loss = history_df.loc[best_epoch, 'val_loss']

            dice_coef = history_df.loc[best_epoch, 'dice_coef']
            val_dice_coef = history_df.loc[best_epoch, 'val_dice_coef']

            iou_coef = history_df.loc[best_epoch, 'iou_coef']
            val_iou_coef = history_df.loc[best_epoch, 'val_iou_coef']

            if not extra_regex:
                all_histories = all_histories.append({'Encoder':regexBone, 'Größe':regexDim, 'Beste Epoche':best_epoch, 'Loss':loss, 'Val. loss':val_loss, 'Dice':dice_coef, 'Val. Dice':val_dice_coef, 'IoU':iou_coef, 'Val. IoU':val_iou_coef}, ignore_index=True)
            else:
                all_histories = all_histories.append({'Encoder':regexBone, 'Größe':regexDim, 'Beste Epoche':best_epoch, 'Loss':loss, 'Val. loss':val_loss, 'Dice':dice_coef, 'Val. Dice':val_dice_coef, 'IoU':iou_coef, 'Val. IoU':val_iou_coef, '2.5D':threeDim, 'Crop':regexCrop,'Bereinigt':regexFaulties ,'Batch Size':regexBatch, 'Epochen':regexEpochs}, ignore_index=True)

    all_histories = all_histories.sort_values(by=['Val. Dice'], ascending=False)
    all_histories.to_csv(output)

    return all_histories

def get_intestines_val(path, output="intestines_vals.csv"):
    all_histories = pd.DataFrame(columns=['Encoder','Größe' ,'Beste Epoche', 'Loss', 'Val. loss', 'Dice', 'Val. Dice', '2.5D', 'Crop' , 'Bereinigt', 'Dickdarm', 'Val. Dickdarm', 'Dünndarm', 'Val. Dünndarm', 'Magen', 'Val. Magen'])

    for file in glob(os.path.join(path, "**"), recursive=True):
        if file.endswith(".csv"):
            print(file)
            history_df = pd.read_csv(file, index_col=0)

            regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties = get_meta_info_from_model_path(file)

            print(regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties)
            
            best_epoch = get_best_epoch(history_df)
            
            loss = history_df.loc[best_epoch, 'loss']
            val_loss = history_df.loc[best_epoch, 'val_loss']
            
            dice_coef = history_df.loc[best_epoch, 'dice_coef']
            val_dice_coef = history_df.loc[best_epoch, 'val_dice_coef']
            
            lb_dice = history_df.loc[best_epoch, 'dice_coef_large_bowel']
            val_lb_dice = history_df.loc[best_epoch, 'val_dice_coef_large_bowel']
            
            sb_dice = history_df.loc[best_epoch, 'dice_coef_small_bowel']
            val_sb_dice = history_df.loc[best_epoch, 'val_dice_coef_small_bowel']
            
            stomach_dice = history_df.loc[best_epoch, 'dice_coef_stomach']
            val_stomach_dice = history_df.loc[best_epoch, 'val_dice_coef_stomach']

            # append every row to the dataframe
            all_histories = all_histories.append({'Encoder':regexBone, 'Größe':regexDim, 'Beste Epoche':best_epoch, 'Loss':loss, 'Val. loss':val_loss, 'Dice':dice_coef, 'Val. Dice':val_dice_coef, '2.5D':threeDim, 'Crop':regexCrop,'Bereinigt':regexFaulties ,'Dickdarm':lb_dice, 'Val. Dickdarm':val_lb_dice, 'Dünndarm':sb_dice, 'Val. Dünndarm':val_sb_dice, 'Magen':stomach_dice, 'Val. Magen':val_stomach_dice}, ignore_index=True)
            
    all_histories = all_histories.sort_values(by=['Val. Dice'], ascending=False)
    all_histories.to_csv(output)

    return all_histories
            
def get_best_epoch(history_df):
    return history_df[history_df["val_loss"]==history_df["val_loss"].min()].index.values[0]

def print_best_epoch(history_df):
    best_epoch = get_best_epoch(history_df)
    print(f"Best epoch: {best_epoch}")
    print(f"Best loss: {history_df.loc[best_epoch, 'val_loss']}")

def print_history_latex(history_df):

    print(history_df.to_latex(index=False , float_format=lambda x: '{:.4f}'.format(x)))

def get_meta_info_from_model_path(file):
    regexBone = 'None' if not re.search("BB_(\w+)_D",file) else re.search("BB_(\w+)_D",file).group(1) 
    regexDim = 'None' if not re.search("DIM_\((\d+),",file) else int(re.search("DIM_\((\d+),",file).group(1) )
    threeDim = 'None' if not re.search("SEMI3D_([aA-zZ]{4,5})_",file) else re.search("SEMI3D_([aA-zZ]{4,5})_",file).group(1) 
    regexCrop = 'None' if not re.search("CROPDATA_([aA-zZ]{4,5})_",file) else re.search("CROPDATA_([aA-zZ]{4,5})_",file).group(1) 
    regexBatch = 'None' if not re.search("BATCH_(\d+)_",file) else int(re.search("BATCH_(\d+)_",file).group(1) )
    regexEpochs = 'None' if not re.search("EPOCHS_(\d+)_",file) else int(re.search("EPOCHS_(\d+)_",file).group(1))
    regexFold = 'None' if not re.search("Fold_(\d).",file) else int(re.search("Fold_(\d).",file).group(1))
    regexFaulties = 'None' if not re.search("FAULTIES_([aA-zZ]{4,5})_",file) else re.search("FAULTIES_([aA-zZ]{4,5})_",file).group(1)

    return regexBone, regexDim, threeDim, regexCrop, regexBatch, regexEpochs, regexFold, regexFaulties


In [13]:
intestines_histories = get_intestines_val("../performance/efb2")

../performance/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_False_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2034_FOLD_3history.csv
efficientnetb2 256 True False 16 50 None False
../performance/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2034_FOLD_3history.csv
efficientnetb2 256 True True 16 50 None False
../performance/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2033_FOLD_3history.csv
efficientnetb2 256 False False 16 50 None True
../performance/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2034_FOLD_3history.csv
efficientnetb2 256 True True 16 50 None True
../performance/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_29082022-2034_FOLD_3history.csv

In [14]:
intestines_histories

,Encoder,Größe,Beste Epoche,Loss,Val. loss,Dice,Val. Dice,2.5D,Crop,Bereinigt,Dickdarm,Val. Dickdarm,Dünndarm,Val. Dünndarm,Magen,Val. Magen
4,efficientnetb2,256,36,0.032584,0.091876,0.924427,0.798258,True,False,True,0.928178,0.827101,0.904222,0.770917,0.93846,0.84119
6,efficientnetb2,256,29,0.034029,0.095001,0.921432,0.790443,False,True,True,0.926124,0.829205,0.899521,0.776028,0.935201,0.850543
3,efficientnetb2,256,31,0.033567,0.095398,0.92261,0.78728,True,True,True,0.926561,0.836502,0.901868,0.772387,0.937739,0.852715
5,efficientnetb2,256,42,0.031538,0.098543,0.927357,0.784954,False,True,False,0.932207,0.809066,0.908839,0.770899,0.938006,0.839136
0,efficientnetb2,256,32,0.032744,0.103691,0.924102,0.769113,True,False,False,0.928257,0.814043,0.903771,0.775286,0.937959,0.816099
2,efficientnetb2,256,26,0.035336,0.107113,0.918085,0.758687,False,False,True,0.921083,0.810628,0.895652,0.742344,0.932458,0.824247
1,efficientnetb2,256,34,0.032208,0.109624,0.925728,0.753002,True,True,False,0.930372,0.805556,0.905696,0.753701,0.937364,0.790431


In [29]:
histories_bs16 =  get_performance_from_history_df(path="../tensorboard_logs/efb2", output="all_histories.csv", extra_regex=True)


../tensorboard_logs/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_25082022-1736_FOLD_3history.csv
efficientnetb2 256 True True 16 50 None True
../tensorboard_logs/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_25082022-1736_FOLD_3history.csv
efficientnetb2 256 True False 16 50 None True
../tensorboard_logs/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_True_CROPDATA_True_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_25082022-1736_FOLD_3history.csv
efficientnetb2 256 True True 16 50 None False
../tensorboard_logs/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_False_FAULTIES_True_BATCH_16_EPOCHS_50_FOLD_3.h5_25082022-1735_FOLD_3history.csv
efficientnetb2 256 False False 16 50 None True
../tensorboard_logs/efb2/UNet_BB_efficientnetb2_DIM_(256, 256, 3)_SEMI3D_False_CROPDATA_True_FAULTIES_False_BATCH_16_EPOCHS_50_FOLD_3.h5_2508202

In [30]:
histories_bs16

,Encoder,Größe,Beste Epoche,Loss,Val. loss,Dice,Val. Dice,IoU,Val. IoU,2.5D,Crop,Bereinigt,Batch Size,Epochen
0,efficientnetb2,256,36,0.030695,0.089621,0.92915,0.802205,0.925499,0.837215,True,True,True,16,50
4,efficientnetb2,256,38,0.032746,0.097435,0.924652,0.787215,0.929342,0.823215,False,True,False,16,50
1,efficientnetb2,256,15,0.040496,0.099924,0.906027,0.777556,0.879747,0.834116,True,False,True,16,50
7,efficientnetb2,256,22,0.037673,0.104097,0.912922,0.768132,0.917503,0.822134,False,False,False,16,50
2,efficientnetb2,256,42,0.030978,0.104837,0.928589,0.765945,0.931438,0.826475,True,True,False,16,50
5,efficientnetb2,256,31,0.034856,0.107887,0.919312,0.757639,0.92184,0.823348,True,False,False,16,50
3,efficientnetb2,256,16,0.041072,0.113215,0.904892,0.742001,0.901741,0.825881,False,False,True,16,50
6,efficientnetb2,256,33,0.037324,0.131027,0.913842,0.696596,0.900161,0.812691,False,True,True,16,50
